<a href="https://www.kaggle.com/code/mastersniffer/reinforcement-learning-custom-environments?scriptVersionId=128829414" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
%pip install stable_baselines3[extra] -q
%pip install gym -q

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [476 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib
      creating build/lib/gym
      copying gym/__init__.py -> build/lib/gym
      copying gym/version.py -> build/lib/gym
      copying gym/logger.py -> build/lib/gym
      copying gym/error.py -> build/lib/gym
      copying gym/core.py -> build/lib/gym
      creating build/lib/gym/spaces
      copying gym/spaces/space.py -> build/lib/gym/spaces
      copying gym/spaces/multi_discrete.py -> build/lib/gym/spaces
      copying gym/spaces/__init__.py -> build/lib/gym/spaces
      copying gym/spaces/discrete.py -> build/lib/gym/spaces
      copying gym/spaces/multi_binary.py -> build/lib/gym/spaces
      copying gym/spaces/tuple.py -> build/lib/gym/spaces
      copying gym/spaces/box.py -> build/lib/gym/spaces
      copying gym/spaces/

In [2]:
import gym 
from gym import Env # This is our super class which will help us make custom env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Types of Spaces

In [3]:
Discrete(3).sample() #Will give from 0 to 2

1

In [4]:
Box(0,1,shape=(3,3)).sample() # Making an array of arrays with numbers from 0 to 1

array([[0.32565215, 0.574794  , 0.8457555 ],
       [0.2509366 , 0.8305495 , 0.91272455],
       [0.78227955, 0.6389102 , 0.66709316]], dtype=float32)

In [5]:
Box(0,1,shape=(3,)).sample() # To make an array of 3 values just leave the second value empty

array([0.7987311 , 0.01409697, 0.41621742], dtype=float32)

In [6]:
Tuple((Discrete(3) , Box(0,1,shape=(5,)))).sample() # Tuple will help us combine different spaces

(0,
 array([0.5930694 , 0.06773072, 0.7496682 , 0.20357746, 0.5075575 ],
       dtype=float32))

In [7]:
print(Dict({'height':Discrete(2), "speed":Box(0,100,shape=(1,))}))
Dict({'height':Discrete(2), "speed":Box(0,100,shape=(1,))}).sample()

Dict(height:Discrete(2), speed:Box([0.], [100.], (1,), float32))


OrderedDict([('height', 0), ('speed', array([51.58663], dtype=float32))])

In [8]:
MultiBinary(4).sample() # it will give us 4 positions

array([0, 0, 0, 0], dtype=int8)

In [9]:
MultiDiscrete([5,5,2]).sample() # The same as binary but discrete
# Value at 1 place varies from 0 to 4, the same for the second value and for the third it varies between 0 and 1

array([2, 2, 1])

# Build Env

- building an agent to give us best shower
- Random temp
- vary between 37 and 39 degrees 

In [10]:
class ShowerEnv(Env): # We need to have 4 key env functions in class: init, step, render, reset
    def __init__(self):
        self.action_space = Discrete(3) # Heat the tub, cool the tub or dont change 
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = 38 + random.randint(-30,30) # Beginning temp is 38 +- 3 degrees (Will change it to bigger value)
        self.shower_length = 120 # How long is shower = 60 (will increase this value)
        pass

    def step(self,action):
        # Apply temp adj
        self.state += action-1 # This is our impact. Why '-1' -> Discrete(3) returns numbers:0,1,2. So for our case 2 means add heat (2 transforms into 1)
                               #, 1 means leave unchanged(1 -> 0) and 0 means to cool our tub 

        # Decrease shower time
        self.shower_length -= 1

        # Calculate reward
        if self.state>=37 and self.state<=39: # Here we make complex of just as simple as that schema where we define our rewards. More complicated are needed for complicated envs !
            reward = 1
        else:
            reward = -1
        
        if self.shower_length<=0: # Here we check that shower is not going negative and if it equals to 0 then we stop the episode
            done=True
        else:
            done=False

        info = {}
        return self.state, reward, done,info

    def render(self):
        # Here we can make view in pygame or other viwes, but i will mark it as TODO
        pass

    def reset(self):
        self.state = np.array([38+random.randint(-30,30)]).astype(float)
        self.shower_length = 120
        return self.state

In [11]:
env = ShowerEnv()

/opt/conda/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


# Test ENV

In [12]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:-120
Episode:2 Score:-120
Episode:3 Score:-120
Episode:4 Score:-92
Episode:5 Score:-112


# Train Model

In [13]:
log_path = os.path.join('Training', 'Logs')


In [14]:
model = PPO('MlpPolicy',env, verbose=0, tensorboard_log=log_path) # Because we have array values we use MlpPolicy

In [15]:
model.learn(total_timesteps=500_000)

# Saving model

In [16]:
shower_path = os.path.join('Training','Saved Models','Shower_Model_PPO')
model.save(shower_path)

/opt/conda/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'Training/Saved Models' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [17]:
del model

In [18]:
model=PPO.load(shower_path,env)

In [19]:
evaluate_policy(model,env, n_eval_episodes=50)

/opt/conda/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(-100.8, 65.11036783800259)